# Lesson 2: Tokenization and Embedding

Welcome to Lesson 2 of our journey to build *MiniLlama2*, a simplified version of a large language model (LLM) inspired by models like Llama 2. In this lesson, we’ll explore two critical preprocessing steps that enable LLMs to understand and process text: **tokenization** and **embedding**. These steps transform raw text—like posts from a forum or a book—into a format that a neural network can work with.

### Why This Matters
Imagine you’re teaching a child to read. First, you break sentences into words (tokenization), then help them understand what those words mean in context (embeddings). LLMs work similarly: they need text split into manageable pieces and then converted into numbers that capture meaning. This lesson lays the foundation for everything that follows in training *MiniLlama2*.

## Objectives
- **Grasp Tokenization**: Learn why and how we split text into smaller units called tokens.
- **Build from Scratch**: Create a basic tokenizer and embedding layer using PyTorch to see the mechanics.
- **Leverage Tools**: Use Hugging Face’s powerful libraries to build a custom tokenizer and tap into pre-trained embeddings.

## Tokenization: Breaking Text Down

Tokenization is the process of splitting text into smaller units, or *tokens*. Think of it as chopping a sentence into pieces a model can digest. There are three main approaches:
- **Word Tokenization**: Splits text by spaces (e.g., "I love coding" → ["I", "love", "coding"]). It’s intuitive for English but fails for languages like Chinese, which don’t use spaces between words.
- **Character Tokenization**: Splits into individual characters (e.g., "cat" → ["c", "a", "t"]). This works for any language but loses higher-level meaning, making it harder for models to learn word relationships.
- **Subword Tokenization**: A hybrid approach that breaks text into meaningful chunks (e.g., "playing" → ["play", "##ing"]). Modern LLMs like Llama 2 use this with algorithms like *Byte Pair Encoding (BPE)* or *WordPiece* to balance vocabulary size and meaning.

For this tutorial, we’re working with a dataset inspired by *Lihkg*, a Hong Kong forum with posts in traditional Chinese. Since Chinese text doesn’t have clear word boundaries, subword tokenization is our best bet. We’ll start simple, then build a proper solution.

## Embedding: Giving Tokens Meaning

Once we have tokens, we need to turn them into numbers because neural networks only understand numerical data. *Embeddings* are dense vectors (e.g., 128 numbers per token) that represent tokens in a way that captures their meaning. For example, "cat" and "dog" might have similar vectors because they’re both pets, while "cat" and "car" would differ more.

We’ll:
- Build a basic embedding layer from scratch to see how it works.
- Use pre-trained embeddings from Hugging Face to show the power of existing solutions.

Let’s dive in with some hands-on coding!

## Implementation

### Part 1: Simple Tokenizer and Embedding
We’ll start with a basic English example to understand the concepts before tackling Chinese text. Using PyTorch, we’ll tokenize a sentence and create embeddings.

In [1]:
import torch

# Simple tokenizer: split by spaces (English example)
text = "Hello world this is a test"
tokens = text.split()  # Splits on whitespace
print("Tokens:", tokens)

# Create a vocabulary: map each unique token to an integer ID
vocab = {token: idx for idx, token in enumerate(set(tokens))}
print("Vocabulary:", vocab)

# Embedding layer: converts token IDs to vectors
embedding_dim = 10  # Each token becomes a 10-number vector
embedding = torch.nn.Embedding(len(vocab), embedding_dim)  # PyTorch’s embedding layer
token_ids = [vocab[token] for token in tokens]  # Convert tokens to IDs
embedded = embedding(torch.tensor(token_ids))  # Get embeddings
print("Embedded vectors shape:", embedded.shape)  # 6 tokens, 10 dimensions each
print("Sample embedding for 'world':", embedded[1])  # Vector for 'world'

Tokens: ['Hello', 'world', 'this', 'is', 'a', 'test']
Vocabulary: {'a': 0, 'test': 1, 'is': 2, 'this': 3, 'world': 4, 'Hello': 5}
Embedded vectors shape: torch.Size([6, 10])
Sample embedding for 'world': tensor([-0.8562, -0.6579,  2.0682, -1.3001, -0.0122, -1.2430, -0.5412, -1.9850,
        -0.3887,  0.4851], grad_fn=<SelectBackward0>)


**What’s Happening Here?**
- We split the sentence into words (tokens).
- We built a *vocabulary*, assigning each unique word a number (e.g., "Hello" = 0).
- The embedding layer randomly initializes a 10-dimensional vector for each token. These vectors will later be tuned during training to capture meaning.
- The output shape `[6, 10]` means 6 tokens, each with a 10-number vector.

This is a toy example. For real models, vocabularies are much larger, and embeddings are pre-trained or learned over massive datasets.

### Part 2: Loading and Tokenizing Lihkg Data

Now, let’s switch to our real dataset: forum posts in traditional Chinese from `notebooks/data/data.csv`. We’ll assume it has a column called `msg` with the text. First, we’ll try a naive approach.

In [2]:
import pandas as pd

# Load the dataset
data_path = 'data/data.csv'
df = pd.read_csv(data_path)
texts = df['head'].tolist()  # Extract the 'msg' column as a list
print("First 5 texts:", texts[:5])

# Simple tokenizer for Chinese (split by spaces, basic)
sample_text = texts[0]  # Take the first post
tokens = sample_text.split()  # Split on spaces
print("Basic tokens (split by space):", tokens)

First 5 texts: ['法國紅酒慢煮阿根廷牛舌 配 煙肉洋蔥炒著仔#wail#pig\\n（$60-5）#wail#pig\\n\\n講吓味道先#wail#pig\\n牛舌有成5大塊#wail#pig\\n已經煮到好淋#wail#pig\\n入口很香，幾乎即化#wail#pig\\n配埋個酸酸地嘅紅酒汁解膩一流#wail#pig\\n同炒著仔都很夾#wail#pig\\n有脷有著有菜#wail#pig\\n又是健康嘅一餐#wail#pig\\n食完條脷根好C伏#wail#pig', '仙台風燒牛舌定食#wail#pig\\n（$63）#wail#pig\\n\\n講吓味道先#wail#pig\\n牛舌佢話有120g#wail#pig\\n大幾5-6片#wail#pig\\n有2舊嘅口感係最好#wail#pig\\n好嫩滑易咬#wail#pig\\n其實4舊口感比較普通#wail#pig\\n燒得幾香，油香唔算勁，但都有#wail#pig\\n醬汁都好入味#wail#pig\\n溏心強整得唔錯，好夠味#wail#pig\\n反而飯係比較差#wail#pig\\n痴埋一舊#wail#pig\\n有脷有菜有飯#wail#pig\\n又是健康嘅一餐#wail#pig\\n食完個吊鐘好C伏#wail#pig', '是#wail#pig', '秦式三餸飯#wail#pig\\n（$47）#wail#pig\\n\\n講吓味道先#wail#pig\\n三餸叫咗豬頸肉，炸髀，肉片炒菜#wail#pig\\n豬頸肉鐵膽唔使講#wail#pig\\n炸髀見佢啱啱拎出嚟#wail#pig\\n麵衣食落仲係香脆#wail#pig\\n但應該唔係生炸，肉質唔算嫩滑多汁#wail#pig\\n炒菜有驚喜，好夠鑊氣，菜都好爽脆#wail#pig\\n似酒樓味嘅炒菜#wail#pig\\n有肉有菜有飯#wail#pig\\n又是健康嘅一餐#wail#pig\\n食完個荷包好C伏#wail#pig#wail#pig', '是#wail#pig']
Basic tokens (split by space): ['法國紅酒慢煮阿根廷牛舌', '配', '煙肉洋蔥炒著仔#wail#pig\\n（$60-5）#wail#pig\\n\\n講吓味道先#wail#pig\\n牛舌有成5大塊#wail

**Why This Isn’t Great**
In English, splitting by spaces works because words are separated that way. In Chinese, text is written without spaces (e.g., "我愛編程" = "I love coding"). Splitting by spaces here might just give us chunks of text, not meaningful words. Let’s fix this with a better approach.

### Part 3: Building a Custom Tokenizer with Hugging Face

For Chinese, we need a smarter tokenizer. We’ll use Hugging Face’s `tokenizers` library to train a *Byte Pair Encoding (BPE)* tokenizer on our dataset. BPE starts with characters and iteratively merges frequent pairs to create subword units—perfect for languages like Chinese.

**Setup Note**: If you haven’t installed `tokenizers`, run this in your terminal or a cell:
```bash
!pip install tokenizers
```

In [3]:
from tokenizers import Tokenizer, models, pre_tokenizers, trainers
import os

# Save texts to a file for training
os.makedirs('temp', exist_ok=True)  # Create a temp directory
with open('temp/train.txt', 'w', encoding='utf-8') as f:
    for text in texts:
        f.write(text + '\n')  # One text per line

# Set up a BPE tokenizer
tokenizer = Tokenizer(models.BPE())  # Use BPE model
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()  # Initial split on whitespace

# Train the tokenizer on our data
trainer = trainers.BpeTrainer(vocab_size=1000, min_frequency=1)  # Limit vocab to 1000 tokens
tokenizer.train(files=['temp/train.txt'], trainer=trainer)  # Learn from our texts

# Save the trained tokenizer
tokenizer.save('temp/custom_tokenizer.json')

# Test it on our sample text
output = tokenizer.encode(sample_text)
print("Custom BPE tokens:", output.tokens)  # Subword tokens
print("Token IDs:", output.ids)  # Numerical IDs

Custom BPE tokens: ['法國紅酒', '慢煮', '阿根廷牛舌', '配', '煙肉', '洋蔥', '炒著仔', '#', 'wail', '#', 'pig', '\\', 'n', '（$', '60', '-', '5', '）#', 'wail', '#', 'pig', '\\', 'n', '\\', 'n講吓味道先', '#', 'wail', '#', 'pig', '\\', 'n牛舌', '有成5大', '塊', '#', 'wail', '#', 'pig', '\\', 'n已', '經煮到好', '淋', '#', 'wail', '#', 'pig', '\\', 'n入', '口很', '香', '，', '幾乎即', '化', '#', 'wail', '#', 'pig', '\\', 'n配', '埋個酸', '酸地嘅紅酒', '汁解', '膩一流', '#', 'wail', '#', 'pig', '\\', 'n同', '炒著仔', '都很夾', '#', 'wail', '#', 'pig', '\\', 'n有脷', '有著', '有菜', '#', 'wail', '#', 'pig', '\\', 'n又是健康嘅一餐', '#', 'wail', '#', 'pig', '\\', 'n食完條脷', '根好C伏', '#', 'wail', '#', 'pig']
Token IDs: [800, 748, 891, 338, 818, 802, 527, 0, 392, 0, 391, 17, 27, 396, 406, 2, 9, 397, 392, 0, 391, 17, 27, 17, 399, 0, 392, 0, 391, 17, 518, 779, 113, 0, 392, 0, 391, 17, 561, 842, 218, 0, 392, 0, 391, 17, 551, 650, 365, 385, 715, 75, 0, 392, 0, 391, 17, 575, 676, 886, 795, 852, 0, 392, 0, 391, 17, 556, 527, 883, 0, 392, 0, 391, 17, 517, 774, 425, 0, 392, 0, 391, 1

**How This Works**
- We save our dataset to a file because the tokenizer needs text input in this format.
- BPE starts with characters and merges frequent pairs (e.g., "我" + "愛" → "我愛") based on our data.
- `vocab_size=1000` limits the vocabulary to 1000 subwords, keeping it small for this demo. Real LLMs use 30,000+.
- The output shows tokens (subwords) and their IDs, ready for embedding.

This custom tokenizer adapts to our Lihkg data, making it ideal for *MiniLlama2*.

### Part 4: Using Pre-trained Embeddings with Hugging Face

Training embeddings from scratch takes time and data. Instead, we can use pre-trained embeddings from a model like BERT, fine-tuned for Chinese. Let’s load `bert-base-chinese`.

**Setup Note**: Install `transformers` if needed:
```bash
!pip install transformers
```

In [4]:
from transformers import BertTokenizer, BertModel

# Load pre-trained Chinese BERT
hf_tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
hf_model = BertModel.from_pretrained('bert-base-chinese')

# Tokenize our sample text
inputs = hf_tokenizer(sample_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
outputs = hf_model(**inputs)  # Pass through BERT
embeddings = outputs.last_hidden_state  # Get the embeddings

print("Pre-trained tokens:", hf_tokenizer.tokenize(sample_text))
print("Embeddings shape:", embeddings.shape)  # [batch_size, seq_length, hidden_size]
print("Sample embedding for first token:", embeddings[0, 0, :5])  # First 5 values

c:\Users\hohoy\OneDrive\Desktop\MiniLlama2-Tutorial\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pre-trained tokens: ['法', '國', '紅', '酒', '慢', '煮', '阿', '根', '廷', '牛', '舌', '配', '煙', '肉', '洋', '蔥', '炒', '著', '仔', '#', 'w', '##ail', '#', 'pi', '##g', '\\', 'n', '（', '$', '60', '-', '5', '）', '#', 'w', '##ail', '#', 'pi', '##g', '\\', 'n', '\\', 'n', '講', '吓', '味', '道', '先', '#', 'w', '##ail', '#', 'pi', '##g', '\\', 'n', '牛', '舌', '有', '成', '5', '大', '塊', '#', 'w', '##ail', '#', 'pi', '##g', '\\', 'n', '已', '經', '煮', '到', '好', '淋', '#', 'w', '##ail', '#', 'pi', '##g', '\\', 'n', '入', '口', '很', '香', '，', '幾', '乎', '即', '化', '#', 'w', '##ail', '#', 'pi', '##g', '\\', 'n', '配', '埋', '個', '酸', '酸', '地', '嘅', '紅', '酒', '汁', '解', '膩', '一', '流', '#', 'w', '##ail', '#', 'pi', '##g', '\\', 'n', '同', '炒', '著', '仔', '都', '很', '夾', '#', 'w', '##ail', '#', 'pi', '##g', '\\', 'n', '有', '[UNK]', '有', '著', '有', '菜', '#', 'w', '##ail', '#', 'pi', '##g', '\\', 'n', '又', '是', '健', '康', '嘅', '一', '餐', '#', 'w', '##ail', '#', 'pi', '##g', '\\', 'n', '食', '完', '條', '[UNK]', '根', '好', '[UNK]', '伏', '#', 

**What’s Going On?**
- `bert-base-chinese` uses a WordPiece tokenizer, breaking text into subwords.
- The model outputs 768-dimensional embeddings (BERT’s `hidden_size`) for each token.
- These embeddings are pre-trained on massive Chinese corpora, capturing rich semantic relationships out of the box.

## Comparing Approaches

Here’s how our methods stack up:
- **Simple Tokenizer**: Quick and easy but useless for Chinese due to no word boundaries.
- **Custom BPE**: Lightweight and tailored to our Lihkg data—great for *MiniLlama2*’s pretraining.
- **Pre-trained BERT**: Heavy-duty with rich embeddings but less flexible and resource-intensive.

For *MiniLlama2*, we’ll use a custom BPE tokenizer in Lesson 6 to keep things efficient and specific to our dataset.

## Exercises: Try It Yourself!

1. **Tokenization Comparison**: Take 10 sentences from `data.csv`. Tokenize them by spaces vs. characters. How do the results differ for Chinese text?
2. **BPE Upgrade**: Train a BPE tokenizer with `vocab_size=2000`. Test it on a new text from the dataset and compare token granularity.
3. **Embedding Exploration**: In the PyTorch embedding layer, try `embedding_dim=128` and `256`. Compute the vector norms (e.g., `torch.norm(embedded, dim=1)`)—what changes?
4. **BERT Visualization**: Tokenize 5 texts with BERT, extract embeddings for the first token of each, and plot their norms (hint: use `matplotlib`). What patterns emerge?

Next up, we’ll build on this to design *MiniLlama2*’s architecture. Happy coding!